In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
from sklearn import metrics
import pandas as pd
import datasets as ds

c:\Users\gasper\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
test = pd.read_csv('../data/headlines_test.csv')#.sample(10, random_state=42)
test2 = pd.read_csv('../data/reddit_test.csv')

dataset = ds.DatasetDict({
    "test": ds.Dataset.from_pandas(test),
    "test2":ds.Dataset.from_pandas(test2)
})

modelname = "EMBEDDIA/sloberta"
tokenizer = AutoTokenizer.from_pretrained(modelname)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=160, truncation=True, return_tensors="pt")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

c:\Users\gasper\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 3000/3000 [00:00<00:00, 10323.79 examples/s]


### Headlines model na headlines testih

In [3]:
model = AutoModelForSequenceClassification.from_pretrained("../training/sloberta_headlines")
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer, device="cuda")
predictions = pipe(dataset["test"]["text"])
prediction = [int(pred['label'].split('_')[-1]) for pred in predictions]

In [4]:
recall = metrics.recall_score(dataset["test"]["label"],prediction)
precision = metrics.precision_score(dataset["test"]["label"],prediction)
f1_score = metrics.f1_score(dataset["test"]["label"],prediction)
accuracy = metrics.accuracy_score(dataset["test"]["label"],prediction)
loss = metrics.log_loss(dataset["test"]["label"],prediction)

print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)

Loss: 5.3272066329827235
Accuracy: 0.8522012578616353
Precision: 0.8787638668779715
Recall: 0.8042059463379261
f1 score: 0.8398333964407422


In [5]:
kappa = metrics.cohen_kappa_score(dataset["test"]["label"],prediction)
print('Cohens kappa: %f' % kappa)
auc = metrics.roc_auc_score(dataset["test"]["label"],prediction)
print('ROC AUC: %f' % auc)
matrix = metrics.confusion_matrix(dataset["test"]["label"],prediction)
print(matrix)

Cohens kappa: 0.703128
ROC AUC: 0.850518
[[1330  153]
 [ 270 1109]]


### Kontekst podmnožica

In [15]:
positive_in = test[test['label'] == 1].sample(15, random_state=42).index
negative_in = test[test['label'] == 0].sample(15, random_state=42).index

c = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

a = [prediction[i] for i in positive_in]
b = [prediction[i] for i in negative_in]
d = a + b

recall = metrics.recall_score(c,d)
precision = metrics.precision_score(c,d)
f1_score = metrics.f1_score(c,d)
accuracy = metrics.accuracy_score(c,d)
loss = metrics.log_loss(c,d)

print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)

Loss: 3.604365338911715
Accuracy: 0.9
Precision: 0.9285714285714286
Recall: 0.8666666666666667
f1 score: 0.896551724137931


### Headlines model na reddit testih

In [16]:
predictions = pipe(dataset["test2"]["text"])
prediction = [int(pred['label'].split('_')[-1]) for pred in predictions]

In [17]:
recall = metrics.recall_score(dataset["test2"]["label"],prediction)
precision = metrics.precision_score(dataset["test2"]["label"],prediction)
f1_score = metrics.f1_score(dataset["test2"]["label"],prediction)
accuracy = metrics.accuracy_score(dataset["test2"]["label"],prediction)
loss = metrics.log_loss(dataset["test2"]["label"],prediction)

print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)

Loss: 18.09391400133681
Accuracy: 0.498
Precision: 0.5069582504970179
Recall: 0.16853932584269662
f1 score: 0.25297619047619047


In [18]:
kappa = metrics.cohen_kappa_score(dataset["test2"]["label"],prediction)
print('Cohens kappa: %f' % kappa)
auc = metrics.roc_auc_score(dataset["test2"]["label"],prediction)
print('ROC AUC: %f' % auc)
matrix = metrics.confusion_matrix(dataset["test2"]["label"],prediction)
print(matrix)

Cohens kappa: 0.001750
ROC AUC: 0.500880
[[1239  248]
 [1258  255]]


### Reddit model na reddit testih

In [4]:
model = AutoModelForSequenceClassification.from_pretrained("../training/sloberta_reddit")
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer, device="cuda")
predictions = pipe(dataset["test2"]["text"])
prediction = [int(pred['label'].split('_')[-1]) for pred in predictions]

In [5]:
recall = metrics.recall_score(dataset["test2"]["label"],prediction)
precision = metrics.precision_score(dataset["test2"]["label"],prediction)
f1_score = metrics.f1_score(dataset["test2"]["label"],prediction)
accuracy = metrics.accuracy_score(dataset["test2"]["label"],prediction)
loss = metrics.log_loss(dataset["test2"]["label"],prediction)

print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)

Loss: 10.80108146560544
Accuracy: 0.7003333333333334
Precision: 0.7114325068870524
Recall: 0.6827495042961005
f1 score: 0.6967959527824621


In [6]:
kappa = metrics.cohen_kappa_score(dataset["test2"]["label"],prediction)
print('Cohens kappa: %f' % kappa)
auc = metrics.roc_auc_score(dataset["test2"]["label"],prediction)
print('ROC AUC: %f' % auc)
matrix = metrics.confusion_matrix(dataset["test2"]["label"],prediction)
print(matrix)

Cohens kappa: 0.400833
ROC AUC: 0.700487
[[1068  419]
 [ 480 1033]]


### Kontekst podmnožica

In [8]:
positive_in = test2[test2['label'] == 1].sample(15, random_state=42).index
negative_in = test2[test2['label'] == 0].sample(15, random_state=42).index

c = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

a = [prediction[i] for i in positive_in]
b = [prediction[i] for i in negative_in]
d = a + b

recall = metrics.recall_score(c,d)
precision = metrics.precision_score(c,d)
f1_score = metrics.f1_score(c,d)
accuracy = metrics.accuracy_score(c,d)
loss = metrics.log_loss(c,d)

print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)

Loss: 8.410185790794003
Accuracy: 0.7666666666666667
Precision: 0.7857142857142857
Recall: 0.7333333333333333
f1 score: 0.7586206896551724


### Reddit model na headlines testih

In [38]:
predictions = pipe(dataset["test"]["text"])
prediction = [int(pred['label'].split('_')[-1]) for pred in predictions]

In [39]:
recall = metrics.recall_score(dataset["test"]["label"],prediction)
precision = metrics.precision_score(dataset["test"]["label"],prediction)
f1_score = metrics.f1_score(dataset["test"]["label"],prediction)
accuracy = metrics.accuracy_score(dataset["test"]["label"],prediction)
loss = metrics.log_loss(dataset["test"]["label"],prediction)

print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)

Loss: 18.96636687771154
Accuracy: 0.47379454926624737
Precision: 0.449802371541502
Recall: 0.4126178390137781
f1 score: 0.43040847201210286


In [40]:
kappa = metrics.cohen_kappa_score(dataset["test"]["label"],prediction)
print('Cohens kappa: %f' % kappa)
auc = metrics.roc_auc_score(dataset["test"]["label"],prediction)
print('ROC AUC: %f' % auc)
matrix = metrics.confusion_matrix(dataset["test"]["label"],prediction)
print(matrix)

Cohens kappa: -0.056866
ROC AUC: 0.471649
[[787 696]
 [810 569]]
